# Лемматизация и очистка скачаных словарей с новостями

In [3]:
import numpy as np
import re

import pandas as pd
import pickle
from tqdm import tqdm_notebook
import time

# Токенизатор 
#from nltk.tokenize import WhitespaceTokenizer
#tokenizer = WhitespaceTokenizer()

from nltk.tokenize import ToktokTokenizer
tokenizer = ToktokTokenizer()

# Лемматизатор 
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

# Стоп-слова 
from nltk.corpus import stopwords
stop = set(stopwords.words('russian'))

In [4]:
# Функция для токенизации, лемматизации и очистки от стоп-слов
def token_lemma(statia, stop = stop):
    # Токенизация 
    out_1 = tokenizer.tokenize(statia.lower())
    # Лемматизация и очистка от стоп-слов
    out_2 = [morph.parse(tx)[0].normal_form for tx in out_1 if tx not in stop]
    # Объединение назад, в текст (Countvectorizer так, сука, хочет, уёбок)
    out_3 = ' '.join(out_2)
    return out_3

In [5]:
!ls ../Сборщик\ новостей/news_data

interfax                president_speach.pickle tass_lemm
interfax_lemm           rbc                     vedomosty
lenta                   ria                     vk_news
lenta_lemm              ria_lemm
old_trio                tass


In [6]:
!mkdir ../Сборщик\ новостей/news_data/lenta_lemm 

mkdir: ../Сборщик новостей/news_data/lenta_lemm: File exists


In [7]:
!ls -lh ../Сборщик\ новостей/news_data/lenta_lemm

total 3009896
-rw-r--r--  1 fulyankin  staff   1,4G 24 ноя 23:17 lenta_news.pickle


In [8]:
files = !ls ../Сборщик\ новостей/news_data/lenta
files = files[:1]
files

['lenta_news.pickle']

In [9]:
files = ['president_speach.pickle']

In [10]:
# Библиотека для распараллеливания кода
from joblib import Parallel, delayed
from tqdm import tqdm_notebook
n_jobs = -1 # параллелим на все ядра 

In [11]:
def prep(nws):
    nws['text'] = token_lemma(nws['text'], stop = stop)
    nws['title'] = token_lemma(nws['title'], stop = stop)
    return nws

In [20]:
# with open('../Сборщик новостей/news_data/tass/{}'.format(files[0]), 'rb') as f:
#     itog_news = pickle.load(f)

In [60]:
for i in tqdm_notebook(files):
    with open('../Сборщик новостей/news_data/lenta/{}'.format(i), 'rb') as f:
        itog_news = pickle.load(f)
    
    # Мало ли! Бывает же... 
    print(time.time())
    print(len(itog_news))
    itog_news  = [itn for itn in itog_news if len(itn) > 0]
    print(len(itog_news))
    
    texts_lemm = Parallel(n_jobs=n_jobs)(delayed(prep)(
        text) for text in tqdm_notebook(itog_news))
        
    with open('../Сборщик новостей/news_data/lenta_lemm/{}'.format(i), 'wb') as f:
        pickle.dump(texts_lemm, f)    

1574613661.609377
464637
464637


In [1]:
!ls -lh ../Сборщик\ новостей/news_data/lenta_lemm

total 3009896
-rw-r--r--  1 fulyankin  staff   1,4G 24 ноя 23:17 lenta_news.pickle


In [2]:
!ls -lh ../Сборщик\ новостей/news_data/lenta

total 3328720
-rw-r--r--  1 fulyankin  staff   1,5G 23 ноя 20:12 lenta_news.pickle
-rw-r--r--  1 fulyankin  staff   101M 23 ноя 13:32 lenta_titles.pickle
